# DeepFM

이번 실습에서는 DeepFM 모델을 이해하고 구현해보겠습니다.  

DeepFM 모델은 Factorization machines와 neural network를 합친 모델로, Wide & Deep model과 유사하지만, feature engineering이 필요하지 않다는 특징을 가지고 있습니다.  
<br/>
사용자가 영화에 대해 Rating한 데이터, 영화의 장르 데이터를 이용하여 Train/Test data를 생성한 다음, Train data로 학습한 모델을 Test data에 대해 평가해봅니다.   
사용한 데이터는 Implicit feedback data로, 사용자가 시청한 영화(Positive instances)는 rating = 1로 기록됩니다. 따라서 시청하지 않은 영화에 대해 각 유저별로 Negative instances sampling을 진행합니다.   
<br/>
**구현에 앞서, DeepFM 논문을 꼭 읽어보시길 권장합니다.**

* 참고  
    - DeepFM: A Factorization-Machine based Neural Network for CTR Prediction (https://arxiv.org/pdf/1703.04247.pdf)  
    - Wide & Deep Learning for Recommender Systems (https://arxiv.org/pdf/1606.07792.pdf)
    - Factorization Machines (https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5694074)
    - https://d2l.ai/chapter_recommender-systems/deepfm.html

# Modules

In [16]:
import csv
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data preprocessing
0. Dataset 다운로드  
<br/>
1. Rating df 생성  
rating 데이터(train_ratings.csv)를 불러와 [user, item, rating]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
2. Genre df 생성   
genre 정보가 담긴 데이터(genres.tsv)를 불러와 genre이름을 id로 변경하고, [item, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.    
<br/>
3. Negative instances 생성   
rating 데이터는 implicit feedback data(rating :0/1)로, positive instances로 구성되어 있습니다. 따라서 rating이 없는 item중 negative instances를 뽑아서 데이터에 추가하게 됩니다.   
<br/>
4. Join dfs   
rating df와 genre df를 join하여 [user, item, rating, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
5. zero-based index로 mapping   
Embedding을 위해서 user,item,genre를 zero-based index로 mapping합니다.
    - user : 0-31359
    - item : 0-6806
    - genre : 0-17  
<br/>
6. feature matrix X, label tensor y 생성   
[user, item, genre] 3개의 field로 구성된 feature matrix를 생성합니다.   
<br/>
7. data loader 생성

## 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [17]:
# 0. Dataset 다운로드
# !wget <대회 데이터 URL>
# !tar -xf data.tar.gz

In [18]:
# 1. Rating df 생성
rating_data = "/opt/ml/input/data/train/train_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)

# 일단 본거는 전부 Positive feedback (1값)으로 처리, 시간 제거
raw_rating_df['rating'] = 1.0 # implicit feedback
raw_rating_df.drop(['time'],axis=1,inplace=True)
# print("Raw rating df")
# print(raw_rating_df)

# user, item 명단 뽑기 (set)
users = set(raw_rating_df.loc[:, 'user'])
items = set(raw_rating_df.loc[:, 'item'])

# 2. Genre df 생성
genre_data = "/opt/ml/input/data/train/genres.tsv"

raw_genre_df = pd.read_csv(genre_data, sep='\t')

#item별 하나의 장르만 남도록 drop
raw_genre_df = raw_genre_df.drop_duplicates(subset=['item']) 
print('-'*50)
print(raw_genre_df)

genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}

# genre 숫자로 변경
raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) 
print("Raw genre df - changed to id")
print(raw_genre_df)

--------------------------------------------------
         item        genre
0         318        Crime
2        2571       Action
5        2959       Action
9         296       Comedy
13        356       Comedy
...       ...          ...
15925   73106       Comedy
15926  109850       Action
15929    8605       Action
15931    3689       Comedy
15932    8130  Documentary

[6807 rows x 2 columns]
Raw genre df - changed to id
         item  genre
0         318      7
2        2571     15
5        2959     15
9         296     17
13        356     17
...       ...    ...
15925   73106     17
15926  109850     15
15929    8605     15
15931    3689     17
15932    8130      6

[6807 rows x 2 columns]


In [19]:
raw_rating_df

,user,item,rating
0,11,4643,1.0
1,11,170,1.0
2,11,531,1.0
3,11,616,1.0
4,11,2140,1.0
...,...,...,...
5154466,138493,44022,1.0
5154467,138493,4958,1.0
5154468,138493,68319,1.0
5154469,138493,40819,1.0


In [20]:
set(list(raw_rating_df.groupby('user')['item'])[1][1])

{1,
 7,
 17,
 62,
 150,
 252,
 260,
 277,
 314,
 339,
 356,
 364,
 368,
 440,
 457,
 468,
 471,
 480,
 497,
 529,
 534,
 552,
 594,
 595,
 728,
 745,
 838,
 898,
 899,
 911,
 924,
 927,
 951,
 953,
 954,
 955,
 1010,
 1013,
 1022,
 1023,
 1042,
 1097,
 1136,
 1148,
 1177,
 1196,
 1197,
 1204,
 1207,
 1210,
 1243,
 1246,
 1262,
 1265,
 1269,
 1270,
 1271,
 1282,
 1291,
 1396,
 1580,
 1610,
 1682,
 1689,
 1947,
 1957,
 2010,
 2012,
 2059,
 2078,
 2083,
 2087,
 2094,
 2105,
 2115,
 2125,
 2150,
 2161,
 2193,
 2294,
 2324,
 2355,
 2394,
 2414,
 2501,
 2687,
 2716,
 2761,
 2797,
 2804,
 2966,
 3088,
 3097,
 3114,
 3175,
 3360,
 3396,
 3397,
 3471,
 3512,
 3548,
 3675,
 3723,
 3751,
 3809,
 3916,
 4022,
 4027,
 4306,
 4366,
 4499,
 4886,
 4896,
 4921,
 4995,
 5066,
 5218,
 5267,
 5299,
 5303,
 5380,
 5470,
 5620,
 5629,
 5667,
 5747,
 5957,
 6297,
 6358,
 6377,
 6424,
 6533,
 6550,
 6662,
 6753,
 6776,
 6936,
 6979,
 7075,
 7161,
 7263,
 7395,
 8039,
 8360,
 8376,
 8529,
 8574,
 8580,
 8633,

In [21]:
# 3. Negative instance 생성
print("Create Nagetive instances")
num_negative = 50
# (user 번호, item dataframe 형식)으로 구조 변경
user_group_dfs = list(raw_rating_df.groupby('user')['item'])

first_row = True
user_neg_dfs = pd.DataFrame()


# u -> user 번호
# u_items -> item 어떤거 봤는지 dataframe
for u, u_items in tqdm(user_group_dfs):
    # u_items 에 겹치는 것 없이 어떤거 봤는지 set로 저장
    u_items = set(u_items)
    
    # 전체 item 에서 유저가 본 u_items를 빼고 
    # 이를 랜덤하게 num_negative (위에서50개로 설정) 개 만큼 선정하여 Negative 항목으로 선정
    # replace :여러번 선택될 수 없음
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    

    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    
    # 첫 번째 항만 예외로 user_neg_dfs생성
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    # 나머지는 뒤에 Concat 하는 방식
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

# 본 항목에 대해 rating 이 1로 되어 있는 raw_rating_df 와
# 보지 않은 50개의 랜덤 항목에 대해 rating이 0으로 되어 있는 user_neg_dfs를 concat
raw_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)

# 4. Join dfs
# negative 항목까지 추가한 raw_rating_df 에 장르 항목까지 Merge
joined_rating_df = pd.merge(raw_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')
# print("Joined rating df")
# print(joined_rating_df)

# 5. user, item을 zero-based index로 mapping
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()

# 위처럼 sort 가 끝났으면 유저, 아이템, 장르가 모두 들어있을 것이므로
# len(users)-1 == max(users) 여야 한다
# 하지만 어쩌다가 빠졌을 수도 있기 때문에 아래와 같은 코드를 작성했다.
# user 수만큼 For문 돌려서 어떤놈이 빠졌는지 
if len(users)-1 != max(users):
    users_dict = dict((pid, i) for (i, pid) in enumerate(pd.unique(joined_rating_df['user'])))
    id2users = dict((i, pid) for (i, pid) in enumerate(pd.unique(joined_rating_df['user'])))
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
  
if len(items)-1 != max(items):
    items_dict = dict((pid, id) for (i, pid) in enumerate(pd.unique(joined_rating_df['item'])))
    id2items = dict((i, pid) for (i, pid) in enumerate(pd.unique(joined_rating_df['item'])))
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

# user 에 맞게 Sort 하고 인덱스도 재설정
joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)

data = joined_rating_df
print("Data")
print(data)

n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)

print("# of data : {}\n# of users : {}\n# of items : {}\n# of genres : {}".format(n_data, n_user, n_item, n_genre))

Create Nagetive instances


100%|██████████| 31360/31360 [03:05<00:00, 169.03it/s]


Data
          user                    item  rating  genre
0            0  <built-in function id>     1.0     15
1            0  <built-in function id>     1.0     11
2            0  <built-in function id>     1.0      0
3            0  <built-in function id>     1.0     15
4            0  <built-in function id>     1.0     15
...        ...                     ...     ...    ...
6722466  31359  <built-in function id>     0.0      4
6722467  31359  <built-in function id>     0.0     15
6722468  31359  <built-in function id>     1.0      9
6722469  31359  <built-in function id>     1.0     17
6722470  31359  <built-in function id>     0.0     17

[6722471 rows x 4 columns]
# of data : 6722471
# of users : 31360
# of items : 1
# of genres : 18


In [22]:
#6. feature matrix X, label tensor y 생성
user_col = torch.tensor(data.loc[:,'user'])
item_col = torch.tensor(data.loc[:,'item'])
genre_col = torch.tensor(data.loc[:,'genre'])

offsets = [0, n_user, n_user+n_item]
for col, offset in zip([user_col, item_col, genre_col], offsets):
    col += offset

X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1)], dim=1)
y = torch.tensor(list(data.loc[:,'rating']))


#7. data loader 생성
# 즁말 간단하고 명확하게 세웠다!
class RatingDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_tensor = input_tensor.long()
        self.target_tensor = target_tensor.long()

    def __getitem__(self, index):
        return self.input_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.target_tensor.size(0)


dataset = RatingDataset(X, y)
train_ratio = 0.9

train_size = int(train_ratio * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

RuntimeError: Could not infer dtype of builtin_function_or_method

   # Model architecture (DeepFM)
   DeepFM 모델은 1) FM component와  2) Deep component가 병렬적으로 결합되어 있습니다. 구조는 다음과 같습니다.
<img src='https://drive.google.com/uc?id=1vwcxUJQTIsg5QH9CuH5PcUEfExhToUHR'>  
각 구조는 다음과 같습니다.  
   **1. FM component**  
       FM component는 우리가 아는 2-way Factorization machines(degree=2)입니다. FM은 variables 간의 interaction을 다음과 같이 모델링 합니다.   
     **<center> equation (1) </center>**
   $$\hat{y}(x):=w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j$$   
   이때, 세번째 interaction term을 전개하여 다음과 같이 쓸 수 있습니다.(논문 참고)  
   구현 코드는 전개된 식을 바탕으로 합니다.   
     **<center> equation (2)> </center>**
   $$\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j = \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2-\sum_{i=1}^{n}v_{i,f}^2x_i^2)$$   
           
   **2. Deep component**  
       Deep component는 MLP Layers로 구성되어 있습니다.   
       구현 코드는 Input dimension이 30-20-10인 3 layer MLP 구조입니다.
  
   

In [ ]:
# input_dims = [n_user, n_item, n_genre]
# embedding_dim = 10
# mlp_dims=[30, 20, 10]

class DeepFM(nn.Module):
    def __init__(self, input_dims, embedding_dim, mlp_dims, drop_rate=0.1):
        super(DeepFM, self).__init__()
        # n_user + n_movie + n_genre
        total_input_dim = int(sum(input_dims)) 

        # Fm component의 constant bias term과 1차 bias term
        self.bias = nn.Parameter(torch.zeros((1,))) #w_0
        self.fc = nn.Embedding(total_input_dim, 1) # w_i * x_i
        
        # 임베딩 term
        self.embedding = nn.Embedding(total_input_dim, embedding_dim) #v_i,f * x_i
        self.embedding_dim = len(input_dims) * embedding_dim

        mlp_layers = []
        for i, dim in enumerate(mlp_dims):
            # mlp_dims 에 맞게 linear 층 쌓아주고
            if i==0:
                mlp_layers.append(nn.Linear(self.embedding_dim, dim))
            else:
                mlp_layers.append(nn.Linear(mlp_dims[i-1], dim))
            # 활성화 함수는 ReLU
            mlp_layers.append(nn.ReLU(True))                     
            # drop out 까즤
            mlp_layers.append(nn.Dropout(drop_rate))
        #마무리로 하나의 답이 나오게 하는 Linear 층 
        mlp_layers.append(nn.Linear(mlp_dims[-1], 1)) 

        # 이를 self.mlp_layers 라 부른다
        self.mlp_layers = nn.Sequential(*mlp_layers)

    # fm 레이어
    def fm(self, x):
        # x : (batch_size, total_num_input)
        embed_x = self.embedding(x) 

        fm_y = self.bias + torch.sum(self.fc(x), dim=1)
        square_of_sum = torch.sum(embed_x, dim=1) ** 2       #TODO 2 : torch.sum을 이용하여 square_of_sum을 작성해주세요(hint : equation (2))
        sum_of_square = torch.sum(embed_x ** 2, dim=1)       #TODO 3 : torch.sum을 이용하여 sum_of_square을 작성해주세요(hint : equation (2))
        fm_y += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        return fm_y
    
    # 딥러닝 레이어
    def mlp(self, x):
        embed_x = self.embedding(x)
        
        inputs = embed_x.view(-1, self.embedding_dim)
        mlp_y = self.mlp_layers(inputs)
        return mlp_y

    def forward(self, x):
        # 임베딩은 각자 모델에서 진행하는 걸로
        
        # fm 조지고
        fm_y = self.fm(x).squeeze(1)
        
        # 딥러닝 조지고
        mlp_y = self.mlp(x).squeeze(1)
        
        # Fm, 딥러닝 값들 더한값을
        # 결과값은 0~1 값이므로 sigmoid 처리
        y = torch.sigmoid(fm_y + mlp_y)
        return y


# Training

In [ ]:
device = torch.device('cuda')
input_dims = [n_user, n_item, n_genre]
embedding_dim = 10
model = DeepFM(input_dims, embedding_dim, mlp_dims=[30, 20, 10]).to(device)
bce_loss = nn.BCELoss() # Binary Cross Entropy loss
lr, num_epochs = 0.01, 10
optimizer = optim.Adam(model.parameters(), lr=lr)

for e in tqdm(range(num_epochs)) :
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        output = model(x)
        loss = bce_loss(output, y.float())
        loss.backward()
        optimizer.step()

100%|██████████| 10/10 [11:36<00:00, 69.66s/it]


# Evaluation
평가는 모델이 postive instance에 대해 0.5이상, negative instance에 대해 0.5미만의 값을 예측한 Accuracy를 측정하여 진행됩니다.

In [ ]:
correct_result_sum = 0
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x)
    result = torch.round(output)
    correct_result_sum += (result == y).sum().float()
with open("DeepFM.pt", 'wb') as f:
    torch.save(model, f)
acc = correct_result_sum/len(test_dataset)*100
print("Final Acc : {:.2f}%".format(acc.item()))

Final Acc : 90.43%


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.



In [ ]:
with open("DeepFM.pt", 'rb') as f:
    model = torch.load(f)

In [ ]:
sub_loader = DataLoader(dataset, batch_size = 10000, shuffle = False)

In [ ]:
#predict
pred = pd.DataFrame()
for x, y in tqdm(sub_loader):
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x) #    
    output = output.detach().cpu().numpy()
    x = x.cpu()
    temp = pd.DataFrame({'user': x[:,0], 'item': x[:,1], 'rating': output})
    pred = pd.concat([pred, temp], axis = 0, sort=False)

100%|██████████| 673/673 [01:43<00:00,  6.48it/s]


In [ ]:
pred = pred.sort_values(['rating','item','user'], ascending=False).reset_index(drop = True)
pred = pred.groupby('user').apply(lambda x : x[:10]).reset_index(drop = True)

In [ ]:

pred['user'] = pred['user'].map(lambda x : id2users[x]) 
pred['item'] = pred['item'].map(lambda x : id2items[x]) 

KeyError: 35981

In [ ]:
pred.drop(labels = 'rating', axis = 1, inplace = True)

In [ ]:
pred.to_csv(os.path.join("../output/", 'DeepFM.csv'), index = False)

In [23]:
pred

,user,item,rating
0,11,35981,0.999773
1,11,34927,0.999744
2,11,34030,0.999731
3,11,34553,0.999712
4,11,36297,0.999704
...,...,...,...
313595,138493,34122,0.999636
313596,138493,36242,0.999489
313597,138493,35981,0.999474
313598,138493,33979,0.999472
